In [45]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from translate import Translator
translator = Translator(to_lang="ar")

In [46]:
#Muharram	Safar	Rabee' Al-Awwal	Rabee' Al-Aakher	Jumadal Al-Aula	Jumadal Al-Aakherah	Rajab	Sha'ban	Ramadan	Shawwal	Dhul Qeadah	Dhul Hijjah
islamic_months = ["muh","saf","raa","rat","jua","jut","raj","sha","ram","shw","kea","hej"]
last_recorded = "shw41"
current = "jua45"
month_to_search = 'kea41'

In [47]:

url = "https://www.astronomycenter.net/icop/"+month_to_search+".html?l=en"
print(url)

headers = {
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
"Accept-Encoding" : "gzip, deflate, br",
"Accept-Language" : "en-GB,en;q=0.9",
"Cache-Control" : "max-age=0",
"Cookie" : "lang=en;",
"Dnt" : "1",
"Referer" : "https://www.astronomycenter.net/res.html",
"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}
cookies = {"lang": "en"}
response = requests.get(url, headers =headers, cookies=cookies)
print(response.status_code)
soup = BeautifulSoup(response.text, 'html.parser')

https://www.astronomycenter.net/icop/kea41.html?l=en


200


In [48]:
table = soup.find('div', class_ = 'container obs')
table = str(table)
countries_h2 = soup.find_all('h2',class_='')[2:-1] #Country elements
country_lines = np.array([c.sourceline for c in countries_h2])

dates_h2 = soup.find_all('h2',class_='required') #Date elements
date_lines = np.array([d.sourceline for d in dates_h2])

table = str(table)

observation_times = []
visibilities = []
location = []
countries = []
obs_dates = []
vis_info = soup.find_all('div', class_ = 'observ') # More than 1 observ per country
for observation in vis_info:

    #Need to get last country_lines number less than value
    observation_line = observation.sourceline
    corresponding_country_index = np.where(country_lines <  observation_line)[0][-1]
    country = countries_h2[corresponding_country_index].text

    #Need to get last country_lines number less than value
    corresponding_date_index = np.where(date_lines <  observation_line)[0][-1]
    date = dates_h2[corresponding_date_index].text

    spans = observation.find_all('span')
    observation_time = spans[0].text.strip()
    visibility = spans[1].text.strip()

    observation_times.append(observation_time)
    visibilities.append(visibility)
    countries.append(country)
    obs_dates.append(date)



cities = re.findall("(?<=from )(.*?)(?= City)",table)
states = re.findall("(?<=City in )(.*?)(?= State)",table)
cloud_levels = re.findall("(?<=the sky was )(.*?)(?=, the atmospheric condition)",table)
atmospheres = re.findall("(?<=atmospheric condition was )(.*?)(?=, the crescent)",table)
v_eye = re.findall("(?<=the crescent was )(.*?)(?= by naked eye,)",table)
v_bino = re.findall("(?<=naked eye, the crescent was )(.*?)(?= by binocular,)",table)
v_telescope = re.findall("(?<=binocular, the crescent was )(.*?)(?= by telescope,)",table)
v_ccd = re.findall("(?<=telescope, the crescent was )(.*?)(?= by CCD Imaging)",table)
islamic_dates = np.full(len(countries), month_to_search)
print(islamic_dates)
print(cities)
print(states)
print(countries)
print(observation_times)
print(visibilities)

print(len(islamic_dates))
print(len(obs_dates))
print(len(cities))
print(len(states))
print(len(countries))
print(len(observation_times))
print(len(cloud_levels))
print(len(observation_times))
print(len(atmospheres))
print(len(visibilities))
print(len(v_eye))
print(len(v_bino))
print(len(v_telescope))
print(len(v_ccd))
print(v_eye)

initial_data = {"Islamic Month": islamic_dates,
                "Date": obs_dates,
                "City": cities,
                 "State": states,
                 "Country":countries,
                   "Obs Time": observation_times,
                   "Cloud Level": cloud_levels,
                   "Atmosphere" : atmospheres,
                     "Visibility":visibilities,
                      "V Eye": v_eye,
                       "V Binocular": v_bino ,
                        "V Telescope": v_telescope ,
                        "V CCD": v_ccd }
data = pd.DataFrame(initial_data)
data.to_csv("..\\Data\\ICOP Updated\\"+month_to_search+".csv")
data.head(100)

['kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41'
 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41'
 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41'
 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41'
 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41' 'kea41']
['حاسي بحبح ', 'قمار', 'كوينين', 'Shahreza', 'الناظور', 'طنجة', 'ارفود', 'تاوريرت', 'بنسليمان ', 'TUCSON', 'Blacksburg', 'الكويف', 'عجاجة', 'حاسي بحبح ', 'باتنة', 'عيسى', 'Dhaka', 'Munich', 'Szedres', 'Surakarta', 'Surakarta', 'Ponorogo', 'Shahreza', 'مدينة الصدر', 'وادي السير', 'Jasin', 'Telok Kemang Observatory, Port Dickson', 'المحمدية', 'بنسليمان ', 'تاوريرت', 'طنجة', 'كاتماندو', 'مقيبلة', 'جدة', 'ABHA', 'الظهران', 'جدة', 'Abu Dhabi', 'York', 'Blacksburg', 'كوينين', 'Ponorogo ', 'Surakarta', 'طنجة']
['الجلفة', 'الوادي', 'الوادي', 'Isfahan', 'الشرق', 'طنجة', 'الراشيدية', 'تاوريرت', 'بنسليمان', 'ARIZONA', 'Virginia', 'تبسة', 'ورقلة'

,Islamic Month,Date,City,State,Country,Obs Time,Cloud Level,Atmosphere,Visibility,V Eye,V Binocular,V Telescope,V CCD
0,kea41,Sun 21 June 2020,حاسي بحبح,الجلفة,Algeria,After sunset,clear,clear,Not Seen,not seen,not sought,not sought,not sought
1,kea41,Sun 21 June 2020,قمار,الوادي,Algeria,After sunset,clear,clear,Not Seen,not seen,not sought,not sought,not sought
2,kea41,Sun 21 June 2020,كوينين,الوادي,Algeria,After sunset,clear,superb,Not Seen,not seen,not seen,not sought,not sought
3,kea41,Sun 21 June 2020,Shahreza,Isfahan,Iran,After sunset,clear,hazy,Not Seen,not sought,not seen,not sought,not sought
4,kea41,Sun 21 June 2020,الناظور,الشرق,Morocco,After sunset,clear,superb,Not Seen,not seen,not sought,not sought,not sought
5,kea41,Sun 21 June 2020,طنجة,طنجة,Morocco,After sunset,clear,superb,Not Seen,not seen,not seen,not sought,not sought
6,kea41,Sun 21 June 2020,ارفود,الراشيدية,Morocco,After sunset,partly cloudy,hazy,Not Seen,not seen,not sought,not sought,not sought
7,kea41,Sun 21 June 2020,تاوريرت,تاوريرت,Morocco,After sunset,clear,clear,Not Seen,not seen,not sought,not sought,not sought
8,kea41,Sun 21 June 2020,بنسليمان,بنسليمان,Morocco,After sunset,clear,superb,Not Seen,not seen,not sought,not sought,not sought
9,kea41,Sun 21 June 2020,TUCSON,ARIZONA,United States,After sunset,clear,clear,Seen,not sought,not sought,seen,not sought
